In [1]:
import math

Diffie-Hellman

In [2]:
def DH_public_key(g, a, p):
    return (g ** a) % p

In [3]:
def DH_shared_secret(public_key, a, p):
    return (public_key ** a) % p

Modulo arithmetics

In [4]:
def inverse(x, p):
    for i in range(1, p):
        if ((x * i) % p) == 1:
            return i
    raise Exception('No inverse exists.')

In [5]:
# Chinese Remainder Theorem
# x = r[i] % n[i]

def CRT(rs, ns):

    N = math.prod(ns)
    x = 0

    for i in range(len(ns)):
        M = int(N / ns[i])
        v = inverse(M, ns[i])
        e = v * M
        x += rs[i] * e

    return x % N

Circle
$ x^{2} + y^{2} = 1 $

In [6]:
def circle_addition(a, b, p):
    x = (a[0] * b[1] + a[1] * b[0]) % p
    y = (a[1] * b[1] - a[0] * b[0]) % p
    return x, y

In [7]:
def DLP_circle(P, P_A, p):
    result = P
    a = 1
    stop = False
    while not stop:
        result = circle_addition(result, P, p)
        if result == P_A:
            stop = True
        a += 1
    return a

Weierstrass curves
General form: $ y^{2} + c_{1}xy + c_{3}y = x^{3} + c_{2}x^{2} + c_{4}x + c_{6} $
Short form: $ y^{2} = x^{3} + c_{4}x + c_{6} $

In [8]:
def general_to_short(cs, p):

    c_1, c_2, c_3, c_4, c_6 = cs[0], cs[1], cs[2], cs[3], cs[4]

    short_c_4 = ((-1 * inverse(48, p) * c_1 ** 4) + (-1 * inverse(6, p) * c_2 * c_1 ** 2) + (inverse(2, p) * c_3 * c_1) + (-1 * inverse(3, p) * c_2 ** 2) + c_4) % p
    short_c_6 = ((inverse(864, p) * c_1 ** 6) + (inverse(72, p) * c_2 * c_1 ** 4) + (-1 * inverse(24, p) * c_3 * c_1 ** 3) + (inverse(18, p) * c_2 ** 2 * c_1 ** 2) +
                 (-1 * inverse(12, p) * c_4 * c_1 ** 2) + (-1 * inverse(6, p) * c_2 * c_3 * c_1) + (2 * inverse(27, p) * c_2 ** 3) + (inverse(4, p) * c_3 ** 2) +
                 (-1 * inverse(3, p) * c_2 * c_4) + c_6) % p

    return [0, 0, 0, short_c_4, short_c_6]

In [9]:
def general_to_short_point(P, cs, p):

    if P is None:
        return P

    c_1, c_2, c_3, c_4, c_6 = cs[0], cs[1], cs[2], cs[3], cs[4]

    y = (P[1] + inverse(2, p) * (c_1 * P[0] + c_3)) % p
    x = (P[0] + inverse(3, p) * (c_2 + inverse(4, p) * c_1 ** 2)) % p

    return x, y

In [10]:
def short_to_general_point(P, cs, p):

    if P is None:
        return P

    c_1, c_2, c_3, c_4, c_6 = cs[0], cs[1], cs[2], cs[3], cs[4]

    y = (P[1] - inverse(2, p) * (c_1 * P[0] + c_3)) % p
    x = (P[0] - inverse(3, p) * (c_2 + inverse(4, p) * c_1 ** 2)) % p

    return x, y

In [11]:
def is_on_weierstrass_curve(P, cs, p):
    x = P[0]
    y = P[1]
    left_side = (y ** 2 + cs[0] * x * y + cs[2] * y) % p
    right_side = (x ** 3 + cs[1] * x ** 2 + cs[3] * x + cs[4]) % p
    return left_side == right_side

In [12]:
def weierstrass_number_of_points(c_4, c_6, p, print_points=False):

    count = 1

    for i in range(p):
        y_squared = ((i ** 3) + c_4 * i + c_6) % p

        for j in range(p):
            if ((j ** 2) % p) == y_squared:
                count += 1

                if print_points:
                    print('(' + str(i) + ', ' + str(j) + ')')
    return count

In [13]:
def weierstrass_order(P, c_4, p):

    sum = P
    order = 1

    while not sum is None:
        sum = weierstrass_addition(sum, P, c_4, p)
        order += 1

    return order

In [14]:
def weierstrass_inverse(P, p):
    if not P is None:
        return P[0], (-1 * P[1]) % p
    else:
        return P

In [15]:
def weierstrass_double(P, c_4, p):

    numerator = (3 * (P[0] ** 2) + c_4) % p
    denominator = (2 * P[1]) % p

    l = (numerator * inverse(denominator, p)) % p

    x = (l ** 2 - 2 * P[0]) % p
    y = (l * (P[0] - x) - P[1]) % p

    return x, y

In [16]:
# P = (x_1, y_1) -> P[0] = x_1, P[1] = y_1
# Q = (x_2, y_2) -> Q[0] = x_2, Q[1] = y_2

def weierstrass_addition_distinct(P, Q, p):

    numerator = (P[1] - Q[1]) % p
    denominator = (P[0] - Q[0]) % p

    l = (numerator * inverse(denominator, p)) % p

    x = (l ** 2 - P[0] - Q[0]) % p
    y = (l * (P[0] - x) - P[1]) % p

    return x, y

In [17]:
# P = (x_1, y_1) -> P[0] = x_1, P[1] = y_1
# Q = (x_2, y_2) -> Q[0] = x_2, Q[1] = y_2

def weierstrass_addition(P, Q, c_4, p):

    if P is None:
        return Q
    if Q is None:
        return P
    if P[0] == Q[0] and P[1] == ((-1 * Q[1]) % p):
        return None
    elif P[0] == Q[0] and P[1] == Q[1]:
        return weierstrass_double(P, c_4, p)
    else:
        return weierstrass_addition_distinct(P, Q, p)

In [18]:
def weierstrass_multiple_additions(P, k, c_4, p):
    Q = None
    for i in range(k):
        Q = weierstrass_addition(Q, P, c_4, p)
    return Q

Weierstrass curves: solving the discrete logarithm problem using the baby-step giant-step algorithm

In [19]:
def BSGS_print(left_side, right_side, a_0, a_1, m, a, P, P_A, c_4, p):

    for i in range(len(left_side)):
        print(str(i) + 'P = ' + str(left_side[i]))
    print('a_0 = ' + str(a_0))
    print()

    for i in range(len(right_side)):
        print('P_A - ' + str(i) + 'mP = ' + str(right_side[i]))
    print('a_1 = ' + str(a_1))
    print()

    print('m = ' + str(m))
    print('a = ' + str(a))
    Q = None
    for i in range(a):
        Q = weierstrass_addition(Q, P, c_4, p)
        print(str(i + 1) + 'P = ' + str(Q))

    print()
    print('P_A = ' + str(P_A))

In [20]:
def DLP_BSGS(P, P_A, c_4, p, print_intermediate=False):

    N = weierstrass_order(P, c_4, p)
    m = math.floor(math.sqrt(N))

    left_side = [None]
    for i in range(m - 1):
        left_side.append(weierstrass_addition(left_side[i], P, c_4, p))

    right_side = [P_A]
    mP = weierstrass_addition(left_side[-1], P, c_4, p)
    n_mP = weierstrass_inverse(mP, p)

    for i in range(math.floor(N / m) + 1):
        right_side.append(weierstrass_addition(right_side[i], n_mP, c_4, p))

        if right_side[-1] is None:
            a_0 = 0
            a_1 = len(right_side) - 1
            a = (a_0 + m * a_1) % N

            if print_intermediate:
                BSGS_print(left_side, right_side, a_0, a_1, m, a, P, P_A, c_4, p)

            return a

        for j in range(1, len(left_side)):
            a = right_side[-1]
            b = left_side[j]

            if a[0] == b[0] and a[1] == b[1]:
                a_0 = j
                a_1 = len(right_side) - 1
                a = (a_0 + m * a_1) % N

                if print_intermediate:
                    BSGS_print(left_side, right_side, a_0, a_1, m, a, P, P_A, c_4, p)

                return a

In [21]:
def BSGS_print_exam(left_side, right_side, a_0, a_1, m, a, P, P_A, cs, p):

    c_4 = general_to_short(cs, p)[-2]

    print('m = ' + str(m))
    print()

    for i in range(len(left_side)):
        print(str(i) + 'R = ' + str(short_to_general_point(left_side[i], cs, p)))
    print('a_0 = ' + str(a_0))
    print()

    for i in range(len(right_side)):
        print('S - ' + str(i) + 'mR = ' + str(short_to_general_point(right_side[i], cs, p)))
    print('a_1 = ' + str(a_1))
    print()

    print('a = ' + str(a))
    Q = None
    for i in range(a):
        Q = weierstrass_addition(Q, P, c_4, p)
        print(str(i + 1) + 'R = ' + str(short_to_general_point(Q, cs, p)))

    print()
    print('S = ' + str(short_to_general_point(P_A, cs, p)))

In [22]:
def DLP_BSGS_exam(P, P_A, cs, p, print_intermediate=False):

    P = general_to_short_point(P, cs, p)
    P_A = general_to_short_point(P_A, cs, p)
    c_4 = general_to_short(cs, p)[-2]

    N = weierstrass_order(P, c_4, p)
    m = math.floor(math.sqrt(N))

    left_side = [None]
    for i in range(m - 1):
        left_side.append(weierstrass_addition(left_side[i], P, c_4, p))

    right_side = [P_A]
    mP = weierstrass_addition(left_side[-1], P, c_4, p)
    n_mP = weierstrass_inverse(mP, p)

    for i in range(math.floor(N / m) + 1):
        right_side.append(weierstrass_addition(right_side[i], n_mP, c_4, p))

        if right_side[-1] is None:
            a_0 = 0
            a_1 = len(right_side) - 1
            a = (a_0 + m * a_1) % N

            if print_intermediate:
                BSGS_print_exam(left_side, right_side, a_0, a_1, m, a, P, P_A, cs, p)

            return a

        for j in range(1, len(left_side)):
            a = right_side[-1]
            b = left_side[j]

            if a[0] == b[0] and a[1] == b[1]:
                a_0 = j
                a_1 = len(right_side) - 1
                a = (a_0 + m * a_1) % N

                if print_intermediate:
                    BSGS_print_exam(left_side, right_side, a_0, a_1, m, a, P, P_A, cs, p)

                return a

Weierstrass curves: solving the discrete logarithm problem using Pollard's rho method

In [23]:
def compute_s(W, k, p):
    return (W[0] % p) % k

In [24]:
def update_W(W, P, P_A, s, c_4, p):
    if s == 0:
        return weierstrass_addition(W, P, c_4, p)
    elif s == 1:
        return weierstrass_addition(W, P_A, c_4, p)
    else:
        return weierstrass_addition(W, W, c_4, p)

In [25]:
def update_b(b, s, p):
    if s == 0:
        return (b + 1) % p
    elif s == 1:
        return b % p
    else:
        return (2 * b) % p

In [26]:
def update_c(c, s, p):
    if s == 0:
        return c % p
    elif s == 1:
        return (c + 1) % p
    else:
        return (2 * c) % p

In [27]:
def step(W, b, c, P, P_A, k, c_4, p):
    s = compute_s(W, k, p)
    return update_W(W, P, P_A, s, c_4, p), update_b(b, s, p), update_c(c, s, p)

In [28]:
def DLP_Pollards_rho_print(Ss, S_bs, S_cs, Fs, F_bs, F_cs, a):

    for i in range(len(Ss)):
        print(
            'S' + str(i) + ' = ' + str(Ss[i]) + '\t\t' + 'b' + str(
            i) + ' = ' + str(S_bs[i]) + '\t\t' + 'c' + str(
            i) + ' = ' + str(S_cs[i])
        )

    print()

    for i in range(len(Fs)):
        print(
            'F' + str(i) + ' = ' + str(Fs[i]) + '\t\t' + 'b' + str(
            i) + ' = ' + str(F_bs[i]) + '\t\t' + 'c' + str(
            i) + ' = ' + str(F_cs[i])
        )

    print()
    print('a = ' + str(a))

In [29]:
def DLP_Pollards_rho(P, P_A, S_0, F_0, b_0, c_0, k, c_4, p, print_intermediate=False):

    Ss = [S_0]
    S_bs = [b_0]
    S_cs = [c_0]

    Fs = [F_0]
    F_bs = [b_0]
    F_cs = [c_0]

    while len(Ss) == 1 or not(
        Ss[-1][0] == Fs[-1][0] and Ss[-1][1] == Fs[-1][1]):

        next_S, next_S_b, next_S_c = step(
            Ss[-1], S_bs[-1], S_cs[-1], P, P_A, k, c_4, p
        )
        Ss.append(next_S)
        S_bs.append(next_S_b)
        S_cs.append(next_S_c)

        next_F, next_F_b, next_F_c = step(
            Fs[-1], F_bs[-1], F_cs[-1], P, P_A, k, c_4, p
        )
        next_F, next_F_b, next_F_c = step(
            next_F, next_F_b, next_F_c, P, P_A, k, c_4, p
        )
        Fs.append(next_F)
        F_bs.append(next_F_b)
        F_cs.append(next_F_c)

    a = ((S_bs[-1] - F_bs[-1]) * inverse(S_cs[-1] - F_cs[-1], p)) % p

    if print_intermediate:
        DLP_Pollards_rho_print(Ss, S_bs, S_cs, Fs, F_bs, F_cs, a)

    return a

Weierstrass curve: solving the discrete logarithm problem using the Pohlig-Hellman attack

In [30]:
def DLP_Pohlig_Hellman(P, Q, c_4, p, N, prime_factorization, print_intermediate=False):

    A = []
    a_is = []

    for i in range(len(prime_factorization)):

        prime_i = prime_factorization[i][0]
        power_i = prime_factorization[i][1]

        Q_i = Q
        A.append([0])
        N_i = int(N / prime_i)
        R_i = weierstrass_multiple_additions(P, N_i, c_4, p)

        if print_intermediate:
            print('i = ' + str(i + 1))
            print('Q_' + str(i + 1) + ' = ' + str(Q_i))
            print('a_' + str(i + 1) + '_-1 = ' + str(A[i][0]))
            print('N_' + str(i + 1) + ' = ' + str(N_i))
            print('R_' + str(i + 1) + ' = ' + str(R_i))
            print()

        for j in range(power_i):

            N_i = int(N / (prime_i ** (j + 1)))

            ap = int(A[i][j] * (prime_i ** (j - 1)))
            apP = weierstrass_multiple_additions(P, ap, c_4, p)
            Q_i = weierstrass_addition(Q_i, weierstrass_inverse(apP, p), c_4, p)

            S_i = weierstrass_multiple_additions(Q_i, N_i, c_4, p)

            A[i].append(DLP_BSGS(R_i, S_i, c_4, p))

            if print_intermediate:
                print('\tj = ' + str(j))
                print('\tN_' + str(i + 1) + ' = ' + str(N_i))
                print('\tQ_' + str(i + 1) + ' = ' + str(Q_i))
                print('\tS_' + str(i + 1) + ' = ' + str(S_i))
                print('\ta_' + str(i + 1) + '_' + str(j) + ' = ' + str(A[i][-1]))
                print()

        a_i = 0
        for j in range(1, len(A[i])):
            a_i += A[i][j] * (prime_i ** (j - 1))
        a_is.append(a_i)

    p_is = []
    for i in range(len(prime_factorization)):
        p_is.append(prime_factorization[i][0] ** prime_factorization[i][1])
        a_is[i] = a_is[i] % p_is[-1]

    if print_intermediate:
        print('a_is = ' + str(a_is))
        print('p_is = ' + str(p_is))

    return CRT(a_is, p_is)

In [31]:
def DLP_Pohlig_Hellman_exam(P, Q, cs, p, N, prime_factorization, print_intermediate=False):

    A = []
    a_is = []

    P = general_to_short_point(P, cs, p)
    Q = general_to_short_point(Q, cs, p)
    c_4 = general_to_short(cs, p)[-2]

    for i in range(len(prime_factorization)):

        prime_i = prime_factorization[i][0]
        power_i = prime_factorization[i][1]

        Q_i = Q
        A.append([0])
        N_i = int(N / prime_i)
        R_i = weierstrass_multiple_additions(P, N_i, c_4, p)

        if print_intermediate:
            print('i = ' + str(i + 1))
            print('p = ' + str(prime_i))
            print('Q_' + str(prime_i) + '_-1 = ' + str(short_to_general_point(Q_i, cs, p)))
            print('a_' + str(prime_i) + '_-1 = ' + str(A[i][0]))
            print('n_' + str(prime_i) + ' = ' + str(N_i))
            print('R_' + str(prime_i) + ' = ' + str(short_to_general_point(R_i, cs, p)))
            print()

        for j in range(power_i):

            N_i = int(N / (prime_i ** (j + 1)))

            ap = int(A[i][j] * (prime_i ** (j - 1)))
            apP = weierstrass_multiple_additions(P, ap, c_4, p)
            Q_i = weierstrass_addition(Q_i, weierstrass_inverse(apP, p), c_4, p)

            S_i = weierstrass_multiple_additions(Q_i, N_i, c_4, p)

            A[i].append(DLP_BSGS(R_i, S_i, c_4, p))

            if print_intermediate:
                print('\tj = ' + str(j))
                print('\tn_' + str(prime_i) + '_' + str(j) + ' = ' + str(N_i))
                print('\tQ_' + str(prime_i) + '_' + str(j) + ' = ' + str(short_to_general_point(Q_i, cs, p)))
                print('\tS_' + str(prime_i) + '_' + str(j) + ' = ' + str(short_to_general_point(S_i, cs, p)))
                print('\ta_' + str(prime_i) + '_' + str(j) + ' = ' + str(A[i][-1]))
                print()

        a_i = 0
        for j in range(1, len(A[i])):
            a_i += A[i][j] * (prime_i ** (j - 1))
        a_is.append(a_i)

    p_is = []
    for i in range(len(prime_factorization)):
        p_is.append(prime_factorization[i][0] ** prime_factorization[i][1])
        a_is[i] = a_is[i] % p_is[-1]

    if print_intermediate:
        print('a_is = ' + str(a_is))
        print('p_is = ' + str(p_is))

    return CRT(a_is, p_is)

RSA

In [32]:
def RSA_key_generation(p, q, e, print_intermediate=False):

    n = p * q
    phi_n = (p - 1) * (q - 1)
    d = inverse(e, phi_n)

    if print_intermediate:
        print('p = ' + str(p))
        print('q = ' + str(q))
        print('e = ' + str(e))
        print('n = ' + str(n))
        print('phi_n = ' + str(phi_n))
        print('d = ' + str(d))

    return d

In [33]:
def RSA_encryption(m, e, n):
    return (m ** e) % n

In [34]:
def RSA_decryption(c, d, n):
    return (c ** d) % n

RSA-CRT

In [35]:
def RSA_CRT_key_generation(p, q, e, print_intermediate=False):

    n = p * q
    phi_n = (p - 1) * (q - 1)
    d = inverse(e, phi_n)
    d_p = d % (p - 1)
    d_q = d % (q - 1)
    u = inverse(p, q)

    if print_intermediate:
        print('p = ' + str(p))
        print('q = ' + str(q))
        print('e = ' + str(e))
        print('n = ' + str(n))
        print('phi_n = ' + str(phi_n))
        print('d = ' + str(d))
        print('d_p = ' + str(d_p))
        print('d_q = ' + str(d_q))
        print('u = ' + str(u))

    return n, p, q, d_p, d_q, u

In [36]:
def RSA_CRT_encryption(m, e, n):
    return RSA_encryption(m, e, n)

In [37]:
def RSA_CRT_decryption(c, p, q, d_p, d_q, u, print_intermediate=False):

    c_p = c % p
    c_q = c % q
    m_p = (c_p ** d_p) % p
    m_q = (c_q ** d_q) % q
    m = (m_p + p * u * (m_q - m_p)) % (p * q)

    if print_intermediate:
        print('c_p = ' + str(c_p))
        print('c_q = ' + str(c_q))
        print('m_p = ' + str(m_p))
        print('m_q = ' + str(m_q))
        print('m = ' + str(m))

    return m

Factorization using the p-1 method

In [38]:
def lcm(A):
    lcm = 1
    for x in A:
        lcm = math.lcm(lcm, x)
    return lcm

In [39]:
def p_1_method(n, a, s, print_intermediate=False):

    s = lcm(s)
    if print_intermediate:
        print('s = ' + str(s))
        print()

    while True:
        b = (a ** s) % n
        d = math.gcd(b - 1, n)

        if print_intermediate:
            print('b = ' + str(b))
            print('d = ' + str(d))
            print()

        if not d == 1 and not d == n:
            return d, int(n / d)